### Define directories and data across conditions
- make sure your data exported from the BVA are in the input directiory according to inputPath variable

In [1]:
import mne
import numpy as np
import glob

inputPath = '/home/koudelka/Projects/LSD_FilipTrybusek/input/'
outputPath = '/home/koudelka/Projects/LSD_FilipTrybusek/output/'
pipePath = '/home/koudelka/Projects/LSD_FilipTrybusek/pipe/'

### Define a list of data to be processed

- the first dimmension is across conditions
- the second dimmension is across diads (groups)
- the third dimmension is across individual subjects within groups
 - in the following exapmple I have one group (one sample), two conditions (EO and EC) and two subjects (S1 and S2)
- define condition names, e.g. ['EO','EC']
 - the order in condLab has to correspond to the order in dataNames 

In [2]:
dataNames = [[['S1_C1_A1_Resting_Segmentation RS EO.vhdr', 'S2_C1_A1_Resting_Segmentation RS EO.vhdr']],
             [['S1_C1_A1_Resting_Segmentation RS EC.vhdr','S2_C1_A1_Resting_Segmentation RS EC.vhdr']]]
dataNames = np.array(dataNames)
condLab = ['EO','EC']

### Example 1
- plot files across all conditions from the first group and the first subject

In [3]:
print(dataNames[:,0,0])

['S1_C1_A1_Resting_Segmentation RS EO.vhdr'
 'S1_C1_A1_Resting_Segmentation RS EC.vhdr']


### Example 2
- plot all subjects in the first condition from the first group

In [4]:
print(dataNames[0,0,:])

['S1_C1_A1_Resting_Segmentation RS EO.vhdr'
 'S2_C1_A1_Resting_Segmentation RS EO.vhdr']


### Init the script and load the data
- specify a band of interest
- specify channels to be dropped
- load the specified data

In [5]:
# bandFreqs = [[1,4],[1,8],[8,12],[12,35],[35,45]]
# bandLabls = ['Delta','Theta','Alpha','Beta','Gamma']
bandFreqs = [8,12]
bandLabels = ['Alpha']
nBands = len(bandFreqs) 
dropChan = ['M1','M2','EOG','ECG','RESP','RESP2']

def init_list(dims, val):
    if len(dims) == 0:
        raise ValueError("Requires at least 1 dimension.")
    if len(dims) == 1:
        return [val for _ in range(dims[0])]
    return [init_list(dims[1:], val=val) for _ in range(dims[0])]

nC,nG,nS = dataNames.shape
dataRaw = init_list(dataNames.shape, val=0)
for cIdx, condIns in enumerate(dataNames):
    for gIdx, groupIns in enumerate(condIns):
        for sIdx, subIns in enumerate(groupIns):
            print('Loading... ' + subIns)
            dataRaw[cIdx][gIdx][sIdx] = mne.io.read_raw_brainvision(inputPath + subIns, verbose='error',preload=True).drop_channels(dropChan,on_missing='ignore')


Loading... S1_C1_A1_Resting_Segmentation RS EO.vhdr
Loading... S2_C1_A1_Resting_Segmentation RS EO.vhdr
Loading... S1_C1_A1_Resting_Segmentation RS EC.vhdr
Loading... S2_C1_A1_Resting_Segmentation RS EC.vhdr


### Rename channels according to subjects ID

In [6]:
#rename channel labels according to subject IDs
for cIdx, condIns in enumerate(dataRaw):
    for gIdx, groupIns in enumerate(condIns):
        for sIdx, subIns in enumerate(groupIns):
            for chIdx, chanIns in enumerate(subIns.ch_names):
                dataRaw[cIdx][gIdx][sIdx].ch_names[chIdx] = chanIns + '_sub' + str(sIdx+1)

### Prepare data for merging group wise
- concatanate across subjects in np array
- generate a new list of channel names

In [7]:
Fs = dataRaw[0][0][0].info['sfreq']
nCh = dataRaw[0][0][0].info['nchan']
newNchan = nCh*nS

#create join np data array and join channel labels out of rawData object
newChanNames = []
npData = init_list((nC,nG),val=0)
for cIdx, condIns in enumerate(dataRaw):
    for gIdx, groupIns in enumerate(condIns):
        for sIdx, subIns in enumerate(groupIns):
            if sIdx == 0:
                npData[cIdx][gIdx] = subIns._data
            else:
                npData[cIdx][gIdx] = np.append(npData[cIdx][gIdx], subIns._data, axis = 0)
            if (cIdx == 0) and (gIdx == 0):
                newChanNames = newChanNames + subIns.ch_names
     
newChanTypes = ["eeg"]*newNchan
newInfo = mne.create_info(newChanNames, ch_types=newChanTypes, sfreq=Fs)

### Create mne raw object out of np array and concatenate across conditions

In [8]:
mneData = init_list((nC,nG),val=0)
for cIdx, condIns in enumerate(npData):
    for gIdx, groupIns in enumerate(condIns):
        mneData[cIdx][gIdx] = mne.io.RawArray(groupIns, newInfo)

Creating RawArray with float64 data, n_channels=64, n_times=120001
    Range : 0 ... 120000 =      0.000 ...   240.000 secs
Ready.
Creating RawArray with float64 data, n_channels=64, n_times=119996
    Range : 0 ... 119995 =      0.000 ...   239.990 secs
Ready.


### Merge bad epochs from single subject annotations 

In [9]:
for cIdx, condIns in enumerate(dataRaw):
    for gIdx, groupIns in enumerate(condIns):        
        annotItem = []
        for sIdx, subIns in enumerate(groupIns):
            for annotIdx, annotIns in enumerate(subIns.annotations.description):
                if 'Bad Interval' in annotIns:
                    annotItem.append([subIns.annotations.onset[annotIdx], subIns.annotations.duration[annotIdx]])     
        #item list to np array
        annotItem = np.array(annotItem) 
        nAnnot = annotItem.shape[0]

        #add the annotations to the data
        mneData[cIdx][gIdx].annotations.append(annotItem[:,0],annotItem[:,1],['bad']*nAnnot)     

### Resample data before further computations
- set new sampling frequency

In [10]:
newFs = 128
for cIdx, condIns in enumerate(mneData):
    for gIdx, groupIns in enumerate(condIns):
        mneData[cIdx][gIdx] = groupIns.resample(newFs)

### Filter data to bands, make epochs, exclude bad epochs
- choose the epoch length
- choose overlap

In [11]:
windowLength = 1 # in seconds
windowOverlap = 0.5
corrList = []

epochData = init_list((nC,nG),val=0)
for cIdx, condIns in enumerate(mneData):
    for gIdx, groupIns in enumerate(condIns):
        mneData[cIdx][gIdx].filter(bandFreqs[0],bandFreqs[1], fir_design='firwin') #alfa
        epochs = mne.make_fixed_length_epochs(mneData[cIdx][gIdx], duration = windowLength, preload=True, reject_by_annotation=True,overlap = windowOverlap)
        epochData = epochs.get_data(copy=True)
        np.savez(pipePath + 'c' + str(cIdx) + 'g' + str(gIdx), epochData=epochData, condLab=condLab)



Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 213 samples (1.664 s)

Not setting metadata
479 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 479 events and 128 original time points ...
232 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) 